In [19]:
import requests
from datetime import datetime, timedelta,timezone

ORION_URL = "http://localhost:1026"
NGSI_LD_ENDPOINT = f"{ORION_URL}/ngsi-ld/v1/entities"
MINTAKA_URL = "http://localhost:8080"

headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/ld+json',
        'Link': '<https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld>; rel="http://www.w3.org/ns/json-ld#context"; type="application/ld+json"'
    }

response = requests.get(f"{ORION_URL}/version")
print(response.json())

response = requests.get(f"{MINTAKA_URL}/info")
print(response.json())

{'orionld version': '1.4.0', 'orion version': '1.15.0-next', 'uptime': '0 d, 0 h, 40 m, 22 s', 'git_hash': 'nogitversion', 'compile_time': 'Sat Aug 26 06:19:09 UTC 2023', 'compiled_by': 'root', 'compiled_in': '', 'release_date': 'Sat Aug 26 06:19:09 UTC 2023', 'doc': 'https://fiware-orion.readthedocs.org/en/master/'}
{'git': {'revision': '9888046fb2bc31d573720cd81f868e6c95866fe7'}, 'build': {'time': '23 February 2022, 07:24:12 +0000'}, 'project': {'artifact-id': 'mintaka', 'group-id': 'org.fiware', 'version': '0.4.3'}}


In [20]:
CP_001_S = {
    "id": "urn:ngsi-ld:ChargingPointStatus:ParisSaclay30-1-P-01-S",
    "type": "ChargingPointStatus",
    "dateCreated": {
        "type": "Property",
        "value": {
            "@type": "DateTime",
            "@value": datetime.now(timezone.utc).isoformat()
        }
    },
    "refChargingPoint": {
        "type": "Relationship",
        "object": "urn:ngsi-ld:ChargingPoint:ParisSaclay30-1-P-01"
    },
    "pointStatus": {
        "type": "Property",
        "value": "available"
    }
}

response = requests.post(
    NGSI_LD_ENDPOINT,
    json=CP_001_S,
    headers=headers
)

print(f"HTTP statu: {response.status_code}")
print(f"info: {response.text}")


HTTP statu: 409
info: {"type":"https://uri.etsi.org/ngsi-ld/errors/AlreadyExists","title":"Entity already exists","detail":"urn:ngsi-ld:ChargingPointStatus:ParisSaclay30-1-P-01-S"}


In [22]:
update_CP_001_S = {
    "pointStatus": {
        "type": "Property",
        "value": "charging"
    }
}

In [23]:
entity_id = "urn:ngsi-ld:ChargingPointStatus:ParisSaclay30-1-P-01-S"

response = requests.patch(
    f"{NGSI_LD_ENDPOINT}/{entity_id}/attrs",
    json=update_CP_001_S,
    headers=headers
)

print(f"HTTP status: {response.status_code}")
print(f"info: {response.text}")

HTTP status: 204
info: 


In [24]:
# Create charging point building 103
def create_charging_point_status(point_number):
    formatted_number = f"{point_number:02d}"
    
    charging_point_status = {
        "id": f"urn:ngsi-ld:ChargingPointStatus:Building103-P-{formatted_number}-S",
        "type": "ChargingPointStatus",
        
        "refChargingPoint": {
            "type": "Relationship",
            "object": f"urn:ngsi-ld:ChargingPointStatus:Building103-P-{formatted_number}"
        },
    
        "pointStatus": {
        "type": "Property",
        "value": "available"
        },  
        
        "dateCreated": {
            "type": "Property",
            "value": {
                "@type": "DateTime",
                "@value": datetime.now().isoformat() + "Z"
            }
        }
    }
    
    return charging_point_status

# Create 10 charging points
charging_point_status = []
for i in range(1, 11):
    cp = create_charging_point_status(i)
    charging_point_status.append(cp)

# Send each charging point to Orion
for i, cp in enumerate(charging_point_status, 1):
    response = requests.post(
        NGSI_LD_ENDPOINT,
        json=cp,
        headers=headers
    )
    
    print(f"charging_point_status {i:02d} - HTTP status: {response.status_code}")
    print(f"charging_point_status {i:02d} - info: {response.text}")

charging_point_status 01 - HTTP status: 409
charging_point_status 01 - info: {"type":"https://uri.etsi.org/ngsi-ld/errors/AlreadyExists","title":"Entity already exists","detail":"urn:ngsi-ld:ChargingPointStatus:Building103-P-01-S"}
charging_point_status 02 - HTTP status: 409
charging_point_status 02 - info: {"type":"https://uri.etsi.org/ngsi-ld/errors/AlreadyExists","title":"Entity already exists","detail":"urn:ngsi-ld:ChargingPointStatus:Building103-P-02-S"}
charging_point_status 03 - HTTP status: 409
charging_point_status 03 - info: {"type":"https://uri.etsi.org/ngsi-ld/errors/AlreadyExists","title":"Entity already exists","detail":"urn:ngsi-ld:ChargingPointStatus:Building103-P-03-S"}
charging_point_status 04 - HTTP status: 409
charging_point_status 04 - info: {"type":"https://uri.etsi.org/ngsi-ld/errors/AlreadyExists","title":"Entity already exists","detail":"urn:ngsi-ld:ChargingPointStatus:Building103-P-04-S"}
charging_point_status 05 - HTTP status: 409
charging_point_status 05 - i